In [3]:
import pyterrier as pt
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
import pandas as pd
from collections import defaultdict

ensure_pyterrier_is_loaded()
tira = Client()

In [5]:
dataset_id = 'longeval-2023-01-20240423-training'
dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset_id}')

In [8]:
offset = '2022'

def keyquery_score_dict(offset, expansion_method, retrieval_model, fb_terms, fb_docs, dataset_id, return_transformer=True):
    f = tira.get_run_output(f'ir-benchmarks/ows/time-keyquery-offset-{offset}', dataset_id)
    f = pd.read_json(f'{f}/{expansion_method}_{retrieval_model}_{fb_terms}_{fb_docs}.jsonl.gz', lines=True)
    default_value = [0, 0]
    ret = defaultdict(lambda: defaultdict(lambda: default_value))

    for _, i in f.iterrows():
        if i['qid'] not in ret:
            ret[i['qid']] = defaultdict(lambda: default_value)
        
        ret[i['qid']][i['docno']] = [1, i['score']]

    return ret

keyquery_dict = keyquery_score_dict(offset, 'bo1', 'BM25', 30, 5, dataset_id)

In [ ]:
# the keyquery dict currently provides vector with two positions:
# - dimension zero is: 0 if the query was not seen before, 1 if the query was seen before
# - dimension one is the score of the document for the query, or 0 if the query was not seen before

In [9]:
#one example of something that is not in the past data:
keyquery_dict['q012318']['doc012308410757']

[0, 0]

In [10]:
#one example of something that is in the reverted index:
keyquery_dict['q0123836']['doc012303707027']

[1, 46.0106946616]